# Name: Khoi Minh Nguyen
# Student ID: 9967930
# COMP28512: Mobile Systems
# Laboratory Task 3

In [ ]:
# Import all the required packages for the task.
import numpy as np
import comp28512_utils as comp 
from scipy import special as sc
import matplotlib.pyplot as plt
import math as mt
from scipy.io import wavfile
from comp28512_utils import get_pesq_scores

## Part 3.1 Effect of increasing transmit power on speech quality

In [ ]:
def get_beP(power):
    # Calculate all the required values.
    # Eb
    power_density = (power / 128.0) * (10.0**-3.0) / 10.0**5.0
    # No
    energy_perBit = 10.0 * (10**-12)
    # Using the formula given.
    return 0.5 * sc.erfc(mt.sqrt(power_density / energy_perBit))

# Clear pesq_results.txt file before calculating PESQ score.
# This is not for the task but to eliminate the warning in the console.
def clear_PESQ():
    f = open("pesq_results.txt","r")
    firstLine = f.readline()
    f.close()
    open("pesq_results.txt","w").close()
    f = open("pesq_results.txt","w")
    f.write(firstLine)
    f.close()

print "Original sound with no bit-errors"
# Read from file NarrobandSpeech8k.
(Fs, y) = wavfile.read("NarrobandSpeech8k.wav")
comp.Audio(y, Fs)

# Converting wave to bit array.
bit_y = comp.numpy_array_to_bit_array(y)

In [ ]:
# For power = 1.0 Watt
print "Sound with bit-errors, for Transmission power = 1.0 Watt"
beP_1 = get_beP(1.0)
print "For beP value:", beP_1
print "And power lost is 50 dB"

# Create some error in the file and convert to numpy array.
y_beP_1 = comp.insert_bit_errors(bit_y, beP_1)
y_1 = comp.bit_array_to_numpy_array(y_beP_1, dtype=np.int16)

comp.Audio(y_1, rate=Fs)

clear_PESQ()

wavfile.write("power_1.wav", Fs, y_1)
! ./linux_pesqmain +8000 NarrobandSpeech8k.wav power_1.wav > /dev/null
# Display PESQ result.
pesq_results = get_pesq_scores()
score = pesq_results["NarrobandSpeech8k.wav"]["power_1.wav"]

print "The PESQ score is:", score

In [ ]:
# For power = 1.35 Watt
print "Sound with bit-errors, for Transmission power = 1.35 Watt"
beP_2 = get_beP(1.35)
print "For beP value:", beP_2
print "And power lost is 50 dB"

# Create some error in the file and convert to numpy array.
y_beP_2 = comp.insert_bit_errors(bit_y, beP_2)
y_2 = comp.bit_array_to_numpy_array(y_beP_2, dtype=np.int16)

comp.Audio(y_2, rate=Fs)

clear_PESQ()

wavfile.write("power_2.wav", Fs, y_2)
! ./linux_pesqmain +8000 NarrobandSpeech8k.wav power_2.wav > /dev/null
# Display PESQ result.
pesq_results = get_pesq_scores()
score = pesq_results["NarrobandSpeech8k.wav"]["power_2.wav"]

print "The PESQ score is:", score

### Part 3.1 Answers

### 1. What is the new bit-error probability and the corresponding PESQ score?
Old bit-error probability: 3.861e-05, with PESQ score: 3.276

New bit-error probability: 2.187e-06, with PESQ score: 4.072

### 2. How is the talk time affected?
Due to the increase in transmit power, the energy consumes will be higher per unit time. Thus the talk time will decrease. Originally, it has 18000 joules in the battery at 1 Watt. Therefore, the talk time will be 18000 / 1 / 3600 = 5 hours of talk time. After the transmit power is increased to 1.35 Watt, the talk time will be 18000 / 1.35 / 3600 = 3.7 hours of total talk time. Thus the talk time decreased by 1.3 hours for a 0.35 Watt of transmitting power increased.

### 3. Explain in about one sentence how battery life and bit-error rates are connected.
Bit-error rate is inversely proportional to the battery life.

## Part 3.2: Effect of increasing bit-error probability on narrow-band speech with and without a (3,1) repetition FEC scheme

In [ ]:
# Function to do majority voting for bit value.
# link: https://en.wikipedia.org/wiki/Majority_function
def fec31_encode(bit_y, beP):
    # Make 3 copies with same bit error probability.
    y_1 = comp.insert_bit_errors(bit_y, beP)
    y_2 = comp.insert_bit_errors(bit_y, beP)
    y_3 = comp.insert_bit_errors(bit_y, beP)

    # Convert value to int for the majority voting algorithm
    y_1 = y_1.astype(int)
    y_2 = y_2.astype(int)
    y_3 = y_3.astype(int)

    result = np.floor(0.5 + ((y_1 + y_2 + y_3 - 0.5) / 3.0)) 
    return result
    
# bit-error graph without (3,1) repitition FEC scheme.
print "==============================================================="
# Output the original file
print "Original sound with no bit-errors."
# Read from file NarrobandSpeech8k.
(Fs, y) = wavfile.read('NarrobandSpeech8k.wav')
comp.Audio(y, Fs)
# Convert it to bit array.
bit_y = comp.numpy_array_to_bit_array(y)
print "==============================================================="

In [ ]:
# Bit-error probability.
x_axis = []
# PESQ value
y_axis = []

# For loop to populate x and y values for the PESQ score to probability graph.
for i in np.arange(-5.0, -2.0 + (5.0 - 2.0) / 13.0, (5.0 - 2.0) / 13.0):
    print "******************************************"
    # Calulating the bit-rate probability.
    current_beP = 10**i
    
    # Add value to the x-axis array.
    x_axis.append(current_beP)
    
    # Create some error in the file and convert to numpy array.
    y_beP = comp.insert_bit_errors(bit_y, current_beP)
    y_error = comp.bit_array_to_numpy_array(y_beP, dtype=np.int16)
    
    print "Sound with bit-error for beP =", current_beP
    comp.Audio(y_error, Fs)

    clear_PESQ()

    # Save file to get PESQ score.
    name = "newfile.wav"
    wavfile.write(name, Fs, y_error)
    ! ./linux_pesqmain +8000 NarrobandSpeech8k.wav newfile.wav > /dev/null
    pesq_results = get_pesq_scores()
    score = pesq_results["NarrobandSpeech8k.wav"][name]
    
    # Add PESQ to the y-axis array.
    y_axis.append(score)
    
    print "The PESQ score is:", score

print "******************************************"

# Plot the graph with x_axis values scale down to log.
plt.plot(x_axis, y_axis)
plt.title("Graph of PESQ score against bit-error probability")
plt.xlabel("Bit-error prob")
plt.xscale("log")
plt.ylabel("PESQ score")
plt.grid()
plt.show()

print "Average PESQ score:", sum(y_axis) / len(y_axis)
print "PESQ score for 10^-2:", y_axis[len(y_axis)-1]

In [ ]:
# bit-error graph with (3,1) repitition FEC scheme.
# Bit-error probability.
x_axis = []
# PESQ value
y_axis = []

# For loop t0 populate x and y values for the PESQ score to probability graph.
for i in np.arange(-5.0, -2.0 + (5.0 - 2.0) / 13.0, (5.0 - 2.0) / 13.0):
    print "******************************************"
    # Calulating the bit-rate probability.
    current_beP = 10**i
    
    # Add value to the x-axis array.
    x_axis.append(current_beP)
    
    # Pick out majority bit using voting.
    y_beP = fec31_encode(bit_y, current_beP)
    y_error = comp.bit_array_to_numpy_array(y_beP, dtype=np.int16)
    
    print "Sound with bit-error for beP =", current_beP
    comp.Audio(y_error, Fs)

    clear_PESQ()

    # Save file to get PESQ score.
    name = "newfile.wav"
    wavfile.write(name, Fs, y_error)
    ! ./linux_pesqmain +8000 NarrobandSpeech8k.wav newfile.wav > /dev/null
    pesq_results = get_pesq_scores()
    score = pesq_results["NarrobandSpeech8k.wav"][name]
    
    # Add PESQ to the y-axis array.
    y_axis.append(score)
    
    print "The PESQ score is:", score

print "******************************************"

# Plot the graph with x_axis values scale down to log.
plt.plot(x_axis, y_axis)
plt.title("Graph of PESQ score against bit-error probability with FEC encoder (3,1)")
plt.xlabel("Bit-error prob")
plt.xscale("log")
plt.ylabel("PESQ score")
plt.grid()
plt.show()

print "Average PESQ score:", sum(y_axis) / len(y_axis)
print "PESQ score for 10^-2:", y_axis[len(y_axis)-1]

### Part 3.2 Answers

### 1. Comment on any improvement in the narrow-band speech quality that is obtained at the expense of the 3-fold increase in the bit-rate.
Firstly, the quality of the sound is much better for values of bit-error probability from 10^-5 to 10^-3.4. The PESQ score for sound produced with probability in the range (from 10^-5 to 10^-3.4) is 4.5. For non FEC encoded sound, it's varied from 4 to 3 PESQ score for the given range. The average PESQ score overall is much higher for the 3-fold increase method 4.0 to 2.4 for non FEC encoded. Furthermore, the sound produced using 3-fold increase in bit rate has less noise and buzzing sound than non FEC encoded sound.

### 2. What is the effect of the 3-fold increase in bit-rate on the transmit power and on the energy required to transmit the speech segment?
The 3-fold increase will cause the transmit power to increase, as more power is required to transfer three copies of a message instead of one. Therefore, the energy required to transmit each speech segment is also increased, this is because there will be 3 segments per speech.

### 3. If we reduce the energy per bit by a factor of 3, the transmit power becomes what it was before? Consider the example in Section 2.5 where E b /N 0 was 8.93 dB. It would now become 8.93 – 10xlog 10 (3) = 8.93 - 4.8 dB= 4.3 dB. From the msk waterfall graph above, the beP now becomes about 0.01. How well does the (3,1) rep coder work at beP=0.01?
For beP = 0.01, the (3,1) rep coder has a PESQ score of 2.0 and the non (3,1) rep coder has a PESQ score of 1.45.

### 4. What other method could be used to reduce the energy required to transmit the speech sentence without reducing the energy per bit? Think about previous Tasks.
By reducing the sampling frequency and bit-rate using uniform quantisation. This means for a constant energy per bit, if we reduce the bit-rate there will be less energy required overall (total) for the message transmission, as it's used a lower bit-rate.

## Part 3.3 Apply ARQ to a text message

In [ ]:
# Function to calculate the crc remainder of a given string.
# This is copied from Wikipedia: https://en.wikipedia.org/wiki/Cyclic_redundancy_check
# The algorithm given in the book is wrong.
def crc_remainder(input_bitstring):
    '''
    Calculates the CRC remainder of a string of bits using a chosen polynomial.
    initial_filler should be '1' or '0'.
    '''
    # Convert the input bool array to a string 
    input_bitstring = np.int8(input_bitstring)
    input_bitstring = np.array_str(input_bitstring)
    input_bitstring = input_bitstring.replace("[","")
    input_bitstring = input_bitstring.replace("]","")
    input_bitstring = input_bitstring.replace(" ","")
    
    # Generator polynomial: g(x) = x^8 + x^2 + x + 1
    polynomial_bitstring = "100000111"

    # initial_filler = 1 as default
    initial_filler = "1"

    len_input = len(input_bitstring)
    initial_padding = initial_filler * (len(polynomial_bitstring) - 1)
    input_padded_array = list(input_bitstring + initial_padding)
    polynomial_bitstring = polynomial_bitstring.lstrip('0')
    while '1' in input_padded_array[:len_input]:
        cur_shift = input_padded_array.index('1')
        for i in range(len(polynomial_bitstring)):
            if polynomial_bitstring[i] == input_padded_array[cur_shift + i]:
                input_padded_array[cur_shift + i] = '0'
            else:
                input_padded_array[cur_shift + i] = '1'
    return ''.join(input_padded_array)[len_input:]

# Function to simulate the message transmission for a given probability.
# Return true for same before and after transmission else false.
def transmission_simulation(beP):
    test_string = "Behold the sea itself, and on its limitless heaving breast, the ships."

    # Convert the string to a bit array.
    bit_string = comp.bytes_to_bit_array(test_string)
    
    # Calculate the original crc value.
    original_crc = crc_remainder(bit_string)
    
    # Add error to the bit array.
    errorBit_string = comp.insert_bit_errors(bit_string, beP)

    # Calculate the error crc value.
    after_crc = crc_remainder(errorBit_string)

    # The string after transmission. 
    string_error = comp.bit_array_to_bytes(errorBit_string)
    
    # Print the string after transmission
    print '"' + string_error + '"'

    if (original_crc == after_crc and test_string == string_error):
        print "Success!!!"
    else:
        print "Fail!!!"

    print "+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+"
    # Return whether success or not.
    return original_crc == after_crc and test_string == string_error

In [ ]:
print "******************************************"
print "Using the string"
print '"Behold the sea itself, and on its limitless heaving breast, the ships."'
print "******************************************"

# Bit-error probability
x_axis = []

# Number of transmission for a succesful message transmit.
y_axis = []

# For loop to calculate the number of re-tranmission for a message, until successful.
for i in np.arange(-4, -0.9, 0.1):
    # Calculate the probability: 10^i
    beP = 10**i
    x_axis.append(beP)
    
    print "============================================================"    
    print "For Probability of", beP
    for j in range(1,10):
        print "Transmission:", j 
        tranmission_success = transmission_simulation(beP)
        if tranmission_success:
            y_axis.append(j)
            break
        # Stopping re-transmission for the 9 re-transmission
        elif j == 9:
            y_axis.append(0)
            print "Stopping re-transmission..."

In [ ]:
# Plot the graph with x_axis values scale down to log.
plt.plot(x_axis, y_axis)
plt.title("Number of transmission for a message with bit-error probability (0 for y means failure to transmiss).")
plt.xlabel("Bit-error probability")
plt.xscale("log")
plt.ylabel("Number of tranmission")
plt.grid()
plt.show()

print "The ARQ process started to fail completely for beP =", x_axis[y_axis.index(0)]

### Part 3.3 Answers

### 1. If the CRC8 check succeeds, can we deduce that there are no bit-errors?
No, we can't guarantee that the transmitted message has no bit-errors. Even though, it's very likely that there are no bit-errors. This is because in my experiment, sometimes the check succeeded, but the output string is completely different to the original string. Thus I added a boolean clause "and test_string == string_error" to make sure that the output string is the same as the original string also.

### 2. If the CRC8 check fails, can we deduce that there are some bit-errors?
Yes, if the check failed it's guarantee (100%) that there is some bit-errors.

### 3. At what beP did the ARQ process fail completely?
The ARQ process started to fail completely for beP = 0.00398107170553

### 4. Do you find this an efficient method? Can you think of a way of improving it by combining failed retransmissions?
The method is efficient for beP value smaller than 0.00398107170553. After that, the method becomes very inefficient as there are many useless re-transmission. A way of improving this method is to store failed transmissions into a buffer every time a transmission fails, then the program will attempt to create the message from these failed transmissions. If the attempt failed, the message is re-transmitted again and the same process will get executed. 